In [2]:
import os
import numpy as np
from pathlib import Path

import pandas as pd
import sys
# kouchou-aiのパスを追加
sys.path.append("../kouchou-ai/server")
sys.path.append("../kouchou-ai/server/broadlistening/pipeline/")

In [3]:
DATA_DIR = Path("../data/original_outputs")  # 広聴AIで出力したデータを保存しているディレクトリ
OUTPUT_DIR = Path("../data/experiment_outputs")

# クラスタリング

In [4]:
from umap import UMAP
from broadlistening.pipeline.steps.hierarchical_clustering import hierarchical_clustering_embeddings

slug = "rep-election-2"
arguments_df = pd.read_csv(DATA_DIR / slug /  "args.csv")
embeddings_df = pd.read_pickle(DATA_DIR / slug / "embeddings.pkl")
embeddings_array = np.asarray(embeddings_df["embedding"].values.tolist())
cluster_nums = [15, 70]
n_samples = embeddings_array.shape[0]
default_n_neighbors = 15

if n_samples <= default_n_neighbors:
    n_neighbors = max(2, n_samples - 1)
else:
    n_neighbors = default_n_neighbors

umap_model = UMAP(random_state=42, n_components=2, n_neighbors=n_neighbors)
umap_embeds = umap_model.fit_transform(embeddings_array)


/Users/nasuka/.pyenv/versions/3.12.4/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/nasuka/.pyenv/versions/3.12.4/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [5]:
# 広聴AIで出力したレポートのslug
cluster_results = hierarchical_clustering_embeddings(
    umap_embeds=umap_embeds,
    cluster_nums=cluster_nums,
)
result_df = pd.DataFrame(
    {
        "arg-id": arguments_df["arg-id"],
        "argument": arguments_df["argument"],
        "x": umap_embeds[:, 0],
        "y": umap_embeds[:, 1],
    }
)
for cluster_level, final_labels in enumerate(cluster_results.values(), start=1):
    result_df[f"cluster-level-{cluster_level}-id"] = [f"{cluster_level}_{label}" for label in final_labels]



start initial clustering
end initial clustering
start hierarchical clustering
[15, 70]
n_cluster_cut:  15
end hierarchical clustering


In [6]:
clusters_argument_df =  result_df.copy()
orig_initial_label_df = pd.read_csv(DATA_DIR / slug / "hierarchical_initial_labels.csv")
orig_merge_label_df = pd.read_csv(DATA_DIR / slug / "hierarchical_merge_labels.csv")

slug_output_dir = OUTPUT_DIR / slug

os.makedirs(slug_output_dir, exist_ok=True)


# プロンプト修正の実験

## 初期ラベリング

In [7]:
INITIAL_PROMPT = """あなたはKJ法が得意なデータ分析者です。userのinputはグループに集まったラベルです。なぜそのラベルが一つのグループであるか解説して、それから表札をつけてください。
出力はJSONとし、フォーマットは以下のサンプルを参考にしてください。


# サンプルの入出力
## 入力例
最近、政治家が能登の復興に向けた具体的なプランを発表し、地域の未来に明るい希望が見えてきました。市民として、真摯な取り組みに感謝しています。
災害復興支援が、選挙期間中にしっかり議論されるようになり、政治家が国民の本当のニーズに応える姿勢に期待しています。
選挙を通じて、政治家が地域振興に全力で取り組む姿勢が伝わってきます。具体的な政策提案を目にするたび、未来への希望が膨らみます。


## 出力例
{{
    "label": "市民の未来を支える具体的政策への期待",
    "description": "このクラスタには、地域の復興や被害者支援など、実際の社会課題に対して政治家が具体的かつ積極的に取り組む姿勢を支持する前向きな意見が集まっています。市民は、選挙や政策議論を通じて、現実の問題に即した支援策や復興計画が実現されることを期待し、明るい未来の構築に向けた政治の変革を応援しています。"
}}"""

In [8]:
from broadlistening.pipeline.steps.hierarchical_initial_labelling import initial_labelling

In [9]:
cluster_id_columns = [col for col in clusters_argument_df.columns if col.startswith("cluster-level-")]
initial_cluster_id_column = cluster_id_columns[-1]
sampling_num = 30
initial_labelling_prompt = INITIAL_PROMPT
model = "gpt-4o"
workers = 70

initial_label_df = initial_labelling(
    initial_labelling_prompt,
    clusters_argument_df,
    sampling_num,
    model,
    workers,
)

In [10]:
# 最下層のクラスタ。プロンプト修正後
initial_label_df.head()

,cluster_id,label,description
0,2_42,衆院選における当選・落選に対する驚きと不満の声,このクラスタは、衆議院選挙における当選や落選に関する驚きや不満の声を中心に構成されています。...
1,2_16,立憲民主党の躍進と自民党の苦戦に関する選挙動向,このクラスタは、立憲民主党が全国的に議席を伸ばし、特に宮城、東京、新潟、福島、長野などで強い...
2,2_61,選挙に対する多様な関心とメディアの影響力,このクラスタは、選挙に対する多様な関心や意見を示す声が集まっています。選挙カーの拡声器に対す...
3,2_20,低投票率に対する懸念と選挙の意義に関する多様な視点,このクラスタは、選挙における低投票率に対する懸念や疑問、そしてその背景にある国民の関心の低さ...
4,2_5,衆議院選挙に対する多様な反応と関心の高まり,このクラスタは、日本の衆議院選挙に対する多様な意見や感情が集まっています。選挙結果に対する期...


In [11]:
initial_clusters_argument_df = clusters_argument_df.merge(
    initial_label_df,
    left_on=initial_cluster_id_column,
    right_on="cluster_id",
    how="left",
).rename(
    columns={
        "label": f"{initial_cluster_id_column.replace('-id', '')}-label",
        "description": f"{initial_cluster_id_column.replace('-id', '')}-description",
    }
)

In [12]:
initial_clusters_argument_df

,arg-id,argument,x,y,cluster-level-1-id,cluster-level-2-id,cluster_id,cluster-level-2-label,cluster-level-2-description
0,Acsv-1_0,当選確実の報道がある一方で、落選確実の状況もあるという意見がある。,9.395892,4.318309,1_15,2_42,2_42,衆院選における当選・落選に対する驚きと不満の声,このクラスタは、衆議院選挙における当選や落選に関する驚きや不満の声を中心に構成されています。...
1,Acsv-2_0,愛媛で自民党が1区と3区、立憲民主党が2区で当選したという意見があるが、政権交代がどう影響す...,6.425354,2.939700,1_9,2_16,2_16,立憲民主党の躍進と自民党の苦戦に関する選挙動向,このクラスタは、立憲民主党が全国的に議席を伸ばし、特に宮城、東京、新潟、福島、長野などで強い...
2,Acsv-3_0,選挙の結果が気になって食事に行けないという声がある。,11.113695,5.199418,1_14,2_61,2_61,選挙に対する多様な関心とメディアの影響力,このクラスタは、選挙に対する多様な関心や意見を示す声が集まっています。選挙カーの拡声器に対す...
3,Acsv-4_0,選挙に行かなかった人に対して消費税を20%にするという意見がある。,9.655325,7.321733,1_13,2_20,2_20,低投票率に対する懸念と選挙の意義に関する多様な視点,このクラスタは、選挙における低投票率に対する懸念や疑問、そしてその背景にある国民の関心の低さ...
4,Acsv-5_0,衆院選が始まり、選挙戦が面白くなってきたという意見がある。,9.691563,4.819920,1_14,2_5,2_5,衆議院選挙に対する多様な反応と関心の高まり,このクラスタは、日本の衆議院選挙に対する多様な意見や感情が集まっています。選挙結果に対する期...
...,...,...,...,...,...,...,...,...,...
10129,Acsv-10085_0,自民党が議席を大幅に減らす可能性があるという意見がある。,3.502974,5.337620,1_8,2_55,2_55,自民党の議席減少とその影響に関する多様な視点,このクラスタは、自民党の議席減少に関する様々な意見や予測を中心に構成されています。自民党の議...
10130,Acsv-10086_0,選挙速報を楽しんで見ているという意見がある。,12.083847,4.712981,1_12,2_40,2_40,選挙速報に対する多様な反応と関心の集まり,このクラスタは、選挙速報に対する人々の多様な反応や関心を示しています。選挙結果に対する期待や...
10131,Acsv-10089_0,死刑囚が選挙権を持たないことに対する意見が述べられている。,10.587885,6.653868,1_13,2_0,2_0,選挙参加の重要性と投票行動に対する意識の多様性,このクラスタは、選挙に参加することの重要性を強調し、投票行動に対する様々な意識や意見が集まっ...
10132,Acsv-10090_0,麻生太郎氏が衆院選で当選したとの情報がある。,6.751141,1.018640,1_11,2_24,2_24,選挙結果に対する多様な反応と情報の信憑性,このクラスタは、衆院選2024における主要候補者の当選確実性や選挙結果に対する多様な意見が集...


In [13]:
initial_clusters_argument_df.to_csv(slug_output_dir / "hierarchical_initial_labels.csv", index=False)

## 統合ラベリング

In [14]:
from broadlistening.pipeline.steps.hierarchical_merge_labelling import _filter_id_columns, merge_labelling, melt_cluster_data, _build_parent_child_mapping, calculate_cluster_density

In [15]:
MERGE_PROMPT = """分割されすぎたクラスタを統合する必要があるので、統合後の名称を考えて出力して。

# 指示
* 統合前のクラスタの名称・説明および統合後のクラスタに属するデータ点のサンプルを与えるので、これらに基づいて統合後のクラスタの名称を出力してください
    * 統合後のクラスタ名において、統合前のクラスタ名をそのまま使うことは避けてください。
* 出力例に記載したJSONのフォーマットに従って出力してください

# サンプルの入出力
## 入力例（クラスタラベル:説明文）
- 地域の災害対応への批判: このクラスタは、地域における災害対応策の実施や体制に対する批判的な意見を集約したものです。住民からは、迅速かつ効果的な支援が行われていない点や、情報提供・連携の不足などに対する強い不満が表明されています。
- 災害対応への不満: このクラスタは、災害発生時の対応全般に対する不満を示す意見をまとめたものです。救援活動の遅れや支援策の実効性に疑問を持つ声が多く、より積極的で透明性のある対応を求める意見が特徴です。
- 地域復興の遅れ: このクラスタは、災害後の地域復興プロセスが予定通りに進んでいない点に対する懸念や不満を反映しています。再建計画や支援策の実施の遅延、そしてそれに伴う住民の生活再建への影響が強調されています。


## 出力例
{{
    "label": "地域再生と災害支援に対する期待と懸念",
    "description": "このクラスタは、特定の地域における再生や災害支援策に対し、具体的な取り組みが不足しているとの意見を集約しています。市民は、選挙を通じた政策議論の中で、地域復興や被災者支援を最優先すべきだとの期待と、現行の支援策に対する改善要求を強く表明しており、より効果的な政府の対応を求める声が反映されています。"
}}"""

In [16]:
config = {}
config["hierarchical_merge_labelling"] = {
    "prompt": MERGE_PROMPT,
    "model": "gpt-4o",
    "sampling_num": 50,
    "workers": 70,
}

In [17]:
clusters_df = initial_clusters_argument_df.copy()
cluster_id_columns: list[str] = _filter_id_columns(clusters_df.columns)
merge_result_df = merge_labelling(
    clusters_df=clusters_df,
    cluster_id_columns=sorted(cluster_id_columns, reverse=True),
    config=config,
)

100%|██████████| 1/1 [00:08<00:00,  8.23s/it]


In [18]:
melted_df = melt_cluster_data(merge_result_df)
parent_child_df = _build_parent_child_mapping(merge_result_df, cluster_id_columns)
melted_df = melted_df.merge(parent_child_df, on=["level", "id"], how="left")
melted_df.head(20)

,level,id,label,description,value,parent
0,1,1_15,衆院選における選挙結果の速報性と有権者の反応,衆議院選挙における当選・落選の速報性に対する驚きや、地域ごとの結果発表のタイミングの違いに対...,691,0
1,1,1_9,2023年衆議院選挙における地域別の政党支持動向と選挙結果の意外性,2023年の衆議院選挙では、地域ごとの政党支持動向が大きく注目されました。立憲民主党が宮城や...,708,0
2,1,1_14,選挙に対する多様な反応とメディアの影響力の分析,選挙に関連する多様な意見や感情が集まり、選挙結果や報道に対する期待、不安、驚き、喜びなどが表...,699,0
3,1,1_13,選挙参加の意識向上と制度改革に関する議論,このクラスタは、選挙に対する国民の関心の低さや投票率の低下に対する懸念、そして選挙制度の改革...,1299,0
4,1,1_4,選挙における政党略称の混乱と議席増加への多様な反応,選挙において、立憲民主党と国民民主党の略称が同じ『民主党』であることが投票の按分に影響を与え...,681,0
5,1,1_10,衆議院選挙結果に対する多様な感情と政治的影響の分析,衆議院選挙の結果に対する有権者の反応は、特定の候補者の当選や落選に対する喜び、驚き、批判、不...,723,0
6,1,1_7,日本の選挙における自民党の支持動向と政治資金問題の影響,このクラスタは、日本の選挙における自民党の支持動向と、政治資金問題が選挙結果に与える影響につ...,603,0
7,1,1_12,選挙報道とスポーツ中継の同時放送における視聴者の反応とメディアの役割,選挙特番とプロ野球日本シリーズの同時放送により、視聴者は情報の混在や画面の見づらさに不満を抱...,807,0
8,1,1_11,選挙結果に対する信憑性の疑念と多様な反応,選挙結果に関する情報の信憑性に対する疑念が広がる中、特定の候補者の当選や落選に対する多様な反...,692,0
9,1,1_3,日本の政治情勢における連立政権の不確実性とその影響,自民党と公明党の連立が過半数を確保できるかどうかに関する不確実性が、日本の政治情勢に大きな影...,973,0


In [19]:
melted_df.to_csv(slug_output_dir / "プロンプト変更後のラベリング結果.csv", index=False)

In [20]:
# 自動評価

In [21]:
import json
import random
from tqdm import tqdm
random.seed(42)

from broadlistening.pipeline.services.llm import request_to_chat_openai

In [28]:
EVALUATION_PROMPT = """あなたはテキストクラスタリングの専門家です。クラスタリング結果のタイトルの適切性を評価してください。

# 指示
与えられた情報に基づいて、クラスタのタイトルがそのクラスタに含まれるテキストデータをどれだけ適切に表現しているかを評価してください。
評価結果は出力例に記載したjson形式で出力してください。

# 入力情報
1. 評価対象クラスタのタイトル
2. 評価対象クラスタに所属するテキストデータのサンプル
3. 他のクラスタからサンプリングしたテキストデータ

# 評価基準
以下の観点から、クラスタのタイトルの適切性を1〜5の5段階で評価してください：

1. inclusiveness_score: タイトルがクラスタ内のすべてのテキストの共通テーマを捉えているか（1: 全く捉えていない 〜 5: 完全に捉えている）
2. specificity_score: タイトルがこのクラスタを他のクラスタと明確に区別できるか（1: 全く区別できない 〜 5: 完全に区別できる）
3. concreteness_score: タイトルが抽象的すぎず、具体的な内容を示しているか（1: 非常に抽象的 〜 5: 非常に具体的）

# 評価例

## 入力例
評価対象クラスタのタイトル: 「災害対応の迅速化と情報共有の強化」

評価対象クラスタに所属するテキストデータのサンプル:
- 災害発生時の初動対応をもっと迅速にしてほしい。特に高齢者への支援が遅れがちだ。
- 避難所の情報がリアルタイムで更新されず、どこに行けばいいのか混乱した。
- 災害時の情報共有システムを強化し、住民が必要な情報にすぐアクセスできるようにすべき。
- 地域ごとの災害対策本部の連携が不十分で、支援物資の配布に無駄が生じていた。
- 災害発生から避難指示が出るまでのタイムラグを短縮する必要がある。

他のクラスタからサンプリングしたテキストデータ:
- 復興予算の使い道をもっと透明化してほしい。どこにいくら使われているのか不明確だ。
- 被災地域の経済復興のための長期的な計画が見えてこない。
- 防災教育を学校カリキュラムに積極的に取り入れるべきだ。
- ボランティアの受け入れ体制が整っておらず、せっかくの支援の手が活かされていない。

## 出力例
{{
  "inclusiveness_score": {{
      "score": 4,
      "reason": "タイトル「災害対応の迅速化と情報共有の強化」は、クラスタ内のテキストの主要なテーマである「初動対応の迅速化」「情報共有の問題」「対策本部の連携」などの要素を広くカバーしています。ただし、「高齢者への支援」という具体的な対象については明示されていないため、完全ではありません。"
  }},
  "specificity_score": {{
      "score": 4,
      "reason": "このタイトルは「対応の迅速化」と「情報共有」という2つの明確な焦点を持っており、他のクラスタ（予算の透明化、経済復興、防災教育など）とは明確に区別できます。ただし、災害対応の中でも「初動」に関する内容が多いため、もう少し特化した表現があるとより区別しやすくなります。"
  }},
  "concreteness_score": {{
      "score": 3,
      "reason": "「迅速化」と「情報共有の強化」という方向性は示されていますが、どのような迅速化（初動対応、避難指示など）なのか、どのような情報共有（避難所情報、支援物資など）なのかまでは具体化されていません。"
  }},
}}
"""

In [29]:
def sample_cluster_data(
    df: pd.DataFrame,
    cluster_column: str,
    cluster_id: str,
    text_column: str,
    sample_size: int
) -> list[str]:
    """クラスタからデータをサンプリングする

    Args:
        df: クラスタリング結果のDataFrame
        cluster_column: クラスタIDが格納されている列名
        cluster_id: サンプリング対象のクラスタID
        text_column: テキストデータが格納されている列名
        sample_size: サンプリングするデータ数

    Returns:
        サンプリングしたテキストデータのリスト
    """
    cluster_data = df[df[cluster_column] == cluster_id]
    sample_size = min(sample_size, len(cluster_data))
    
    if sample_size == 0:
        return []
    
    return cluster_data.sample(sample_size)[text_column].tolist()


def sample_other_clusters_data(
    df: pd.DataFrame,
    cluster_column: str,
    exclude_cluster_id: str,
    text_column: str,
    sample_size: int,
    sample_clusters: int | None = None
) -> list[str]:
    """他のクラスタからデータをサンプリングする

    Args:
        df: クラスタリング結果のDataFrame
        cluster_column: クラスタIDが格納されている列名
        exclude_cluster_id: 除外するクラスタID
        text_column: テキストデータが格納されている列名
        sample_size: 各クラスタからサンプリングするデータ数
        sample_clusters: サンプリングするクラスタ数（Noneの場合は全クラスタ）

    Returns:
        サンプリングしたテキストデータのリスト
    """
    other_clusters = df[df[cluster_column] != exclude_cluster_id][cluster_column].unique()
    
    if len(other_clusters) == 0:
        return []
    
    if sample_clusters is not None and sample_clusters < len(other_clusters):
        other_clusters = random.sample(list(other_clusters), sample_clusters)
    
    sampled_data = []
    for cluster_id in other_clusters:
        cluster_samples = sample_cluster_data(
            df, cluster_column, cluster_id, text_column, sample_size
        )
        sampled_data.extend(cluster_samples)
    
    return sampled_data


def evaluate_cluster_title(
    prompt: str,
    cluster_title: str,
    cluster_samples: list[str],
    other_clusters_samples: list[str],
    model: str = "gpt-4o"
) -> dict[str, dict]:
    """クラスタのタイトルを評価する

    Args:
        prompt: 評価用プロンプト
        cluster_title: 評価対象のクラスタタイトル
        cluster_samples: 評価対象クラスタのサンプルデータ
        other_clusters_samples: 他のクラスタのサンプルデータ
        model: 使用するLLMモデル名

    Returns:
        評価結果（JSON形式）
    """
    # 入力データの整形
    input_text = f"評価対象クラスタのタイトル: 「{cluster_title}」\n\n"
    
    input_text += "評価対象クラスタに所属するテキストデータのサンプル:\n"
    for sample in cluster_samples:
        input_text += f"- {sample}\n"
    
    input_text += "\n他のクラスタからサンプリングしたテキストデータ:\n"
    for sample in other_clusters_samples:
        input_text += f"- {sample}\n"
    
    # LLMに評価リクエスト
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": input_text},
    ]
    
    try:
        response = request_to_chat_openai(messages=messages, model=model, is_json=True)
        evaluation = json.loads(response)
        return evaluation
    except Exception as e:
        print(f"評価中にエラーが発生しました: {e}")
        return {
            "error": str(e),
            "title_evaluation": {
                "inclusiveness_score": {"score": 0, "reason": "評価エラー"},
                "specificity_score": {"score": 0, "reason": "評価エラー"},
                "concreteness_score": {"score": 0, "reason": "評価エラー"},
                "overall_score": {"score": 0, "reason": "評価エラー"},
            }
        }

def evaluate(
    prompt: str,
    clusters_df: pd.DataFrame,
    cluster_column: str,
    text_column: str,
    sample_size: int,
    other_sample_size: int,
    other_sample_clusters: int | None = None
) -> list[dict[str, dict]]:
    cluster_ids = sorted(clusters_df[cluster_column].unique())
    evaluation_results = []
    for cluster_id in tqdm(cluster_ids):
        cluster_title = clusters_df[clusters_df[cluster_column] == cluster_id]["cluster-level-2-label"].values[0]
        cluster_samples = sample_cluster_data(clusters_df, cluster_column, cluster_id, text_column, sample_size)
        other_clusters_samples = sample_other_clusters_data(clusters_df, cluster_column, cluster_id, text_column, other_sample_size, other_sample_clusters)
        evaluation_result = evaluate_cluster_title(prompt, cluster_title, cluster_samples, other_clusters_samples)
        evaluation_results.append(evaluation_result)
    return evaluation_results


In [30]:
cluster_column = "cluster-level-1-id"
cluster_ids = sorted(clusters_df[cluster_column].unique())


In [31]:
eval_results = evaluate(
    prompt=EVALUATION_PROMPT,
    clusters_df=clusters_df,
    cluster_column="cluster-level-1-id",
    text_column="argument",
    sample_size=30,
    other_sample_size=4,
    other_sample_clusters=10
)

100%|██████████| 15/15 [01:30<00:00,  6.04s/it]


In [32]:
eval_score_df = pd.DataFrame([{key: val_dict["score"] for key, val_dict in eval_result.items()} for eval_result in eval_results])

In [33]:
eval_score_df.mean(axis=0)

inclusiveness_score    4.2
specificity_score      3.6
concreteness_score     3.6
dtype: float64

In [34]:
eval_score_df

,inclusiveness_score,specificity_score,concreteness_score
0,5,4,4
1,5,4,3
2,5,4,4
3,3,4,4
4,4,3,3
5,4,3,3
6,5,4,4
7,5,4,4
8,5,4,4
9,2,3,3
